In [4]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [5]:
import tensorflow as tf
import numpy as np
import os, time
import traceback
from datetime import datetime
from tqdm import tqdm
from infolog import log
from utils import ValueWindow

from tensorflow.errors import OutOfRangeError
from feeder_wav import Feeder
from resnet import ResNet
from vox12_hparams import train_feeder_hparams, dev_feeder_hparams, resnet_hparams, train_hparams

In [3]:
#!pip install GPUtil

In [6]:
def create_model(fbanks, labels):
    with tf.variable_scope('resnet', reuse=tf.AUTO_REUSE):
        train_resnet = ResNet(resnet_hparams, fbanks, labels, 'train')
        train_resnet.build_graph()
    with tf.variable_scope('resnet', reuse=tf.AUTO_REUSE):
        eval_resnet = ResNet(resnet_hparams, fbanks, labels, 'eval')
        eval_resnet.build_graph()

    return train_resnet, eval_resnet

# pretrain model path
model_dir = 'vox12_resnet34_pretrain/'

graph = tf.Graph()
with graph.as_default():
    fbanks = tf.placeholder(dtype=tf.float32, shape=(None, None, 80), name='fbanks')
    labels = tf.placeholder(dtype=tf.int32, shape=(None), name='labels')
    _, resnet = create_model(fbanks, labels)
    saver = tf.train.Saver()
    

checkpoint_state = tf.train.get_checkpoint_state(model_dir)
checkpoint_path = checkpoint_state.model_checkpoint_path

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(graph=graph, config=config)

saver.restore(sess, checkpoint_path)
    


INFO:tensorflow:Restoring parameters from vox12_resnet34_pretrain/gvector.ckpt-300000


## Obtain speaker embedding for dataset VCTK

In [7]:
pairs = []
save_dir = '/home/marinjezidzic/Downloads/tf_multispeakerTTS_fc/datasets/syn_accent/synthesizer/embeds/'
mel_dir = '/home/marinjezidzic/Downloads/tf_multispeakerTTS_fc/datasets/syn_accent/synthesizer/mels/'
os.makedirs(save_dir, exist_ok=True)
with open('../datasets/syn_accent/synthesizer/train.txt') as f:
    for line in f.readlines():
        items = line.strip().split('|')
        pairs.append((mel_dir + items[1], save_dir + items[2]))

In [8]:
for mel_file, embed_file in tqdm(pairs):
    try:
        mels = np.load( str(mel_file) )
        print('loaded')
    except:
        continue;
    embedding = sess.run(resnet.gv, feed_dict={fbanks: np.expand_dims(mels, axis=0)})
    embedding = embedding.squeeze()
    np.save(embed_file, embedding)
    print('saved')

  0%|                                                                                                                | 0/2137 [00:00<?, ?it/s]

loaded


  0%|                                                                                                        | 2/2137 [00:00<08:43,  4.08it/s]

saved
loaded
saved
loaded


  0%|▏                                                                                                       | 4/2137 [00:00<04:52,  7.29it/s]

...

saved
loaded
saved
loaded
saved
loaded
saved
loaded


100%|████████████████████████████████████████████████████████████████████████████████████████████████████▉| 2135/2137 [02:00<00:00, 20.06it/s]

saved
loaded
saved
loaded
saved
loaded
saved
loaded
saved
loaded


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 2137/2137 [02:00<00:00, 17.66it/s]

saved
loaded
saved


In [16]:
np.load('/home/marinjezidzic/Downloads/tf_multispeakerTTS_fc/datasets/vctk/synthesizer/mels/mel-p227_154.npy')

array([[-2.59499  , -3.0558915, -3.451487 , ..., -3.791201 , -3.8498108,
        -4.       ],
       [-2.905467 , -3.3732247, -3.7478497, ..., -4.       , -4.       ,
        -4.       ],
       [-3.1103776, -3.363897 , -3.7261877, ..., -4.       , -4.       ,
        -4.       ],
       ...,
       [-3.1005533, -3.4526477, -3.7083604, ..., -4.       , -4.       ,
        -4.       ],
       [-2.7767198, -3.537266 , -3.4125817, ..., -4.       , -4.       ,
        -4.       ],
       [-2.7635858, -3.1976743, -3.144128 , ..., -4.       , -4.       ,
        -4.       ]], dtype=float32)